# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,qaanaaq,77.4840,-69.3632,-21.41,57,18,3.56,GL,1699936121
1,1,kysyl-syr,63.8986,122.7617,-12.47,95,100,2.20,RU,1699936122
2,2,guerrero negro,27.9769,-114.0611,20.15,60,100,4.18,MX,1699935962
3,3,raudeberg,61.9875,5.1352,4.93,1,4,2.68,NO,1699936123
4,4,batagay,67.6333,134.6333,-21.03,90,100,2.48,RU,1699936124


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot=city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles=True
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp_df=city_data_df[(city_data_df['Max Temp']<26.6667) & (city_data_df['Max Temp']>12.7778)]

# Drop any rows with null values
ideal_temp_df=ideal_temp_df.dropna()

# Display sample data
ideal_temp_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,guerrero negro,27.9769,-114.0611,20.15,60,100,4.18,MX,1699935962
7,7,castro,-24.7911,-50.0119,20.61,67,19,0.58,BR,1699936126
14,14,waitati,-45.7500,170.5667,16.56,65,90,2.32,NZ,1699936130
16,16,dunedin,-45.8742,170.5036,16.56,65,100,2.68,NZ,1699935970
18,18,avarua,-21.2078,-159.7750,23.03,64,40,3.09,CK,1699936132


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=city_data_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,qaanaaq,77.4840,-69.3632,-21.41,57,18,3.56,GL,1699936121,
1,1,kysyl-syr,63.8986,122.7617,-12.47,95,100,2.20,RU,1699936122,
2,2,guerrero negro,27.9769,-114.0611,20.15,60,100,4.18,MX,1699935962,
3,3,raudeberg,61.9875,5.1352,4.93,1,4,2.68,NO,1699936123,
4,4,batagay,67.6333,134.6333,-21.03,90,100,2.48,RU,1699936124,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
qaanaaq - nearest hotel: Hotel Qaanaaq
kysyl-syr - nearest hotel: No hotel found
guerrero negro - nearest hotel: Plaza sal paraiso
raudeberg - nearest hotel: No hotel found
batagay - nearest hotel: Киселях
punta arenas - nearest hotel: Hotel Lacolet
mahebourg - nearest hotel: Grand Bel Air
castro - nearest hotel: Hotel Central Palace
saint-philippe - nearest hotel: Le Baril
yellowknife - nearest hotel: No hotel found
shelburne - nearest hotel: No hotel found
muravlenko - nearest hotel: Е-Отель
butaritari - nearest hotel: No hotel found
bengkulu - nearest hotel: Dena Hotel
waitati - nearest hotel: No hotel found
prince rupert - nearest hotel: Crest Hotel
dunedin - nearest hotel: Scenic Hotel Dunedin City
georgetown - nearest hotel: Page 63 hostel
avarua - nearest hotel: Paradise Inn
hilo - nearest hotel: Dolphin Bay Hotel
tiksi - nearest hotel: Арктика
bethel - nearest hotel: Hampton Inn Danbury
rikitea - nearest hotel: Chez Bianca & Benoit
sabang - nearest hotel: 

odweyne - nearest hotel: فندق المدينه
pandan - nearest hotel: No hotel found
kloulklubed - nearest hotel: The Adventures Inn
lebu - nearest hotel: No hotel found
shenavan - nearest hotel: No hotel found
ilhabela - nearest hotel: Vila Kebaya
tchibanga - nearest hotel: Auberge Nyanga
taltal - nearest hotel: Hostería Taltal
tuatapere - nearest hotel: No hotel found
bantry - nearest hotel: Bantry Bay Hotel
broome - nearest hotel: No hotel found
namibe - nearest hotel: Hotel Chik Chik Namibe
gryazovets - nearest hotel: Дружба
pareora - nearest hotel: No hotel found
suez - nearest hotel: فندق لوسيندا
upernavik - nearest hotel: No hotel found
atar - nearest hotel: فندق سكليل
san patricio - nearest hotel: No hotel found
pangnirtung - nearest hotel: Auyuittuq Lodge
broken hill - nearest hotel: Ibis Styles
kpandu - nearest hotel: Catherine’s Lodge
makaha - nearest hotel: No hotel found
rocha - nearest hotel: Hotel Municipal
lusambo - nearest hotel: No hotel found
bredasdorp - nearest hotel: Vict

jieshi - nearest hotel: No hotel found
portland - nearest hotel: the Hoxton
ekhabi - nearest hotel: Сфера
kita - nearest hotel: Hotel Chats Rouges
hillsboro - nearest hotel: Residence Inn Portland Hillsboro/Brookwood
chulym - nearest hotel: No hotel found
saskylakh - nearest hotel: No hotel found
kohlu - nearest hotel: No hotel found
panama city - nearest hotel: Hostal Familiar Carmencita
wanning - nearest hotel: No hotel found
ulaangom - nearest hotel: АНДУУД
clyde river - nearest hotel: No hotel found
naze - nearest hotel: ビジネス旅館畠山
glyadyanskoye - nearest hotel: No hotel found
jambi - nearest hotel: Hotel Odua Weston Jambi
manado - nearest hotel: Swiss Belhotel
kigoma - nearest hotel: sunset vista hotel
lagunas - nearest hotel: Hospedaje Eco
joshimath - nearest hotel: Snow Crest Hotel
yashan - nearest hotel: No hotel found
bilecik - nearest hotel: Namlı Osmanlı Otel
borovoy - nearest hotel: No hotel found
khandyga - nearest hotel: Апельсин
vilhena - nearest hotel: No hotel found
kras

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,qaanaaq,77.4840,-69.3632,-21.41,57,18,3.56,GL,1699936121,Hotel Qaanaaq
1,1,kysyl-syr,63.8986,122.7617,-12.47,95,100,2.20,RU,1699936122,No hotel found
2,2,guerrero negro,27.9769,-114.0611,20.15,60,100,4.18,MX,1699935962,Plaza sal paraiso
3,3,raudeberg,61.9875,5.1352,4.93,1,4,2.68,NO,1699936123,No hotel found
4,4,batagay,67.6333,134.6333,-21.03,90,100,2.48,RU,1699936124,Киселях
...,...,...,...,...,...,...,...,...,...,...,...
544,544,ponta delgada,37.7333,-25.6667,18.40,88,20,1.79,PT,1699936534,Hotel Talisman
545,545,port lincoln,-34.7333,135.8667,17.28,67,53,4.68,AU,1699936536,Boston Hotel
546,546,nelson bay,-32.7167,152.1500,27.03,79,52,10.28,AU,1699936536,No hotel found
547,547,tacuarembo,-31.7333,-55.9833,15.88,98,100,2.83,UY,1699936537,Hotel Carlos Gardel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot 
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo=True, 
    size="Humidity", 
    scale=1,
    color="City",
    alpha= 0.5,
    tiles=True,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)